# **Поиск наилучших гиперпараметров модели YOLOv8 с помощью метода KFolds**

## Подключение Google Disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Установка необходимых библиотек

In [ ]:
!pip install ultralytics
!pip install scikit-learn
import cv2
import os
from matplotlib import pyplot as plt
import shutil
from ultralytics import YOLO
import yaml
from sklearn.model_selection import KFold

## Применение метода k-folds

### Гиперпараметры для обучения
y_type - размер модели. "n" - самая маленькая, "s" - средняя, "m" - самая большая.

y_epochs - количество эпох для обучения на каждом фолде

y_batch - размер батча

y_size - разрешение изображений

k - количество фолдов

### Пути к папкам с изображениями и метками

In [ ]:
images_path = "/content/drive/MyDrive/stem_det/dataset/images"
labels_path = "/content/drive/MyDrive/stem_det/dataset/labels"

### Гиперпараметры для обучения

In [ ]:
y_type = 'n'
y_epochs = 150
y_batch = 16
y_size = 640
k = 4

In [ ]:
# Создаем список всех изображений и меток
all_images = sorted(os.listdir(images_path))
all_labels = sorted(os.listdir(labels_path))

# Проверяем, что количество изображений совпадает с количеством меток
assert len(all_images) == len(all_labels), "Количество изображений и меток не совпадает!"

# Инициалиализация KFold
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Список для хранения результатов каждого fold
fold_results = []

# Путь к предобученной модели YOLOv8
pretrained_model_path = f"yolov8{y_type}.pt"

# k-fold
for fold, (train_idx, val_idx) in enumerate(kf.split(all_images)):
    print(f"Fold {fold + 1}/{k}")

    # Создаем временные папки для текущего fold
    fold_images_train = f"/content/drive/MyDrive/stem_det/images/train"
    fold_images_val = f"/content/drive/MyDrive/stem_det/images/val"
    fold_labels_train = f"/content/drive/MyDrive/stem_det/labels/train"
    fold_labels_val = f"/content/drive/MyDrive/stem_det/labels/val"

    os.makedirs(fold_images_train, exist_ok=True)
    os.makedirs(fold_images_val, exist_ok=True)
    os.makedirs(fold_labels_train, exist_ok=True)
    os.makedirs(fold_labels_val, exist_ok=True)

    # Перемещаем файлы для тренировки
    for idx in train_idx:
        shutil.copy(os.path.join(images_path, all_images[idx]), fold_images_train)
        shutil.copy(os.path.join(labels_path, all_labels[idx]), fold_labels_train)

    # Перемещаем файлы для валидации
    for idx in val_idx:
        shutil.copy(os.path.join(images_path, all_images[idx]), fold_images_val)
        shutil.copy(os.path.join(labels_path, all_labels[idx]), fold_labels_val)

    # Генерация .yaml файла для текущего fold
    fold_data_yaml_path = f"/content/drive/MyDrive/stem_det/fold_{fold}_data.yaml"
    fold_data = {
        "train": fold_images_train,
        "val": fold_images_val,
        "nc": 2,
        "names": ["background_top", "target_top"]
    }
    with open(fold_data_yaml_path, "w") as yaml_file:
        yaml.dump(fold_data, yaml_file, default_flow_style=False)

    # Загрузка предобученных весов модели YOLO
    print(f"Загрузка предобученной модели: {pretrained_model_path}")
    model = YOLO(pretrained_model_path)

    # Обучение модели на текущем fold
    results = model.train(
        data=fold_data_yaml_path,
        epochs=y_epochs,
        imgsz=y_size,
        batch=y_batch,
        conf=0.3,
        iou=0.25,
        lr0=0.005,
        optimizer="Adam",
        name=f"yolov8{y_type}_{y_epochs}_{y_batch}_{y_size}_fold{fold + 1}_2class",
        val=True
    )

    # Сохраняем путь к лучшей модели текущего фолда
    best_model_path = f"runs/detect/yolov8{y_type}_{y_epochs}_{y_batch}_{y_size}_fold{fold + 1}_2class/weights/best.pt"
    print(f"Лучшие веса для Fold {fold + 1} сохранены: {best_model_path}")

    # Сохраняем метрики для каждого класса
    metrics = {
        "precision": results.box.p.tolist(),
        "recall": results.box.r.tolist(),
        "ap50": results.box.ap50.tolist(),
        "ap": results.box.ap.tolist()
    }

    fold_results.append(metrics)

    # Выводим метрики для каждого класса
    class_names = results.names
    for class_id, class_name in class_names.items():
        print(f"Класс {class_id} ({class_name}):")
        print(f"  Precision: {metrics['precision'][class_id]:.4f}")
        print(f"  Recall: {metrics['recall'][class_id]:.4f}")
        print(f"  AP@0.5: {metrics['ap50'][class_id]:.4f}")
        print(f"  AP@[0.5:0.95]: {metrics['ap'][class_id]:.4f}")

    # Удаляем временные папки, чтобы не занимать лишнее место
    shutil.rmtree(fold_images_train)
    shutil.rmtree(fold_images_val)
    shutil.rmtree(fold_labels_train)
    shutil.rmtree(fold_labels_val)

# Усреднение метрик по фолдам для каждого класса
average_metrics = {
    "precision": [],
    "recall": [],
    "ap50": [],
    "ap": []
}

num_classes = len(fold_results[0]["precision"])  # Количество классов

for class_id in range(num_classes):
    avg_precision = sum([fold["precision"][class_id] for fold in fold_results]) / k
    avg_recall = sum([fold["recall"][class_id] for fold in fold_results]) / k
    avg_ap50 = sum([fold["ap50"][class_id] for fold in fold_results]) / k
    avg_ap = sum([fold["ap"][class_id] for fold in fold_results]) / k

    average_metrics["precision"].append(avg_precision)
    average_metrics["recall"].append(avg_recall)
    average_metrics["ap50"].append(avg_ap50)
    average_metrics["ap"].append(avg_ap)

# Вывод усредненных метрик по каждому классу
print("\nУсредненные метрики по всем фолдам:")
for class_id, class_name in class_names.items():
    print(f"Класс {class_id} ({class_name}):")
    print(f"  Precision: {average_metrics['precision'][class_id]:.4f}")
    print(f"  Recall: {average_metrics['recall'][class_id]:.4f}")
    print(f"  AP@0.5: {average_metrics['ap50'][class_id]:.4f}")
    print(f"  AP@[0.5:0.95]: {average_metrics['ap'][class_id]:.4f}")


Fold 1/4
Загрузка предобученной модели: yolov8n.pt


100%|██████████| 6.25M/6.25M [00:00<00:00, 197MB/s]

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/stem_det_kfolds_2klass/fold_0_data.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_150_16_640_fold1_2class, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.3, iou=0.25, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384,

100%|██████████| 5.35M/5.35M [00:00<00:00, 87.7MB/s]


...

    150/150      2.33G      1.234      1.025      1.179         18        640: 100%|██████████| 4/4 [00:01<00:00,  2.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.36it/s]

                   all         16        133      0.731      0.521        0.6      0.266



150 epochs completed in 0.262 hours.
Optimizer stripped from runs/detect/yolov8n_150_16_640_fold4_2class/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_150_16_640_fold4_2class/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_150_16_640_fold4_2class/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


                   all         16        133      0.785       0.55      0.665      0.272
        background_top         16         74      0.811      0.405      0.583      0.229
            target_top         16         59      0.759      0.695      0.746      0.316
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/yolov8n_150_16_640_fold4_2class
Лучшие веса для Fold 4 сохранены: runs/detect/yolov8n_150_16_640_fold4_2class/weights/best.pt
Класс 0 (background_top):
  Precision: 0.8108
  Recall: 0.4054
  AP@0.5: 0.5828
  AP@[0.5:0.95]: 0.2288
Класс 1 (target_top):
  Precision: 0.7593
  Recall: 0.6949
  AP@0.5: 0.7463
  AP@[0.5:0.95]: 0.3160

Усредненные метрики по всем фолдам:
Класс 0 (background_top):
  Precision: 0.7191
  Recall: 0.2678
  AP@0.5: 0.4754
  AP@[0.5:0.95]: 0.2048
Класс 1 (target_top):
  Precision: 0.7925
  Recall: 0.7040
  AP@0.5: 0.7843
  AP@[0.5:0.95]: 0.4237


In [ ]:
# Укажите путь к папке, которую хотите скачать
folder_path = "/content/runs/detect"  # Путь к папке с результатами

# Упаковываем папку в архив .zip
shutil.make_archive('/content/results_backup', 'zip', folder_path)

'/content/results_backup.zip'